In [ ]:
import sys
sys.path.append('./fdnn_balatsoukas') # for balatsoukas plot func


import tensorflow as tf
from tensorflow import keras
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tensorflow.python.ops import math_ops
from models import NeuralNetGlobalHammer, NeuralNetGlobalWiener, NeuralNetGlobalHammerWiener, FFNN_model, cmplxMeanSquaredError
from util import compute_GMACS
from config import *

import pickle
import fullduplex as fd #balatsoukas

In [ ]:
CHANNEL_LEN = 13
model_order = ['FFNN', 'H', 'W', 'WH']
models = [
    FFNN_model(CHANNEL_LEN, nHidden=17),
    NeuralNetGlobalHammer(filterLen=CHANNEL_LEN),
    NeuralNetGlobalWiener(filterLen=CHANNEL_LEN),
    NeuralNetGlobalHammerWiener(filterLen=CHANNEL_LEN)
]
tensorSpec_shapes = [(30244, 26), (1, 1, TOTAL_SIGNAL_LENGTH, 1), (1, 1, TOTAL_SIGNAL_LENGTH, 1), (1, 1, TOTAL_SIGNAL_LENGTH, 1)]
tensorSpec_dtypes = [tf.float64, tf.complex128, tf.complex128, tf.complex128]

print('Model \t GMACs \t\t #Parameters')
print('------------------------------------')
for ii in range(len(model_order)):
    model = models[ii]    
    model.compile(loss=cmplxMeanSquaredError(),    
              optimizer=keras.optimizers.Adam(learning_rate=0.003, amsgrad=False, beta_1=0.9) , # Einzelsignal 0.01,  Zehn Signale lr=0.001 bei batchsize=1
              metrics=['MeanSquaredError'], run_eagerly=True)
    
    total_params = sum([np.prod(v.shape) for v in model.trainable_variables])
    gmacs = compute_GMACS(model, tensorSpec_shapes[ii], tensorSpec_dtypes[ii])

    print(model_order[ii], '\t', '' f'{gmacs:.5f}', '\t', total_params)